<a href="https://colab.research.google.com/github/lautitorraca/7506R-2C2023-GRUPO24/blob/main/TP2/7506R_TP2_GRUPO24_LIMPIEZA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 𝔾𝕣𝕦𝕡𝕠 𝟚𝟜 (Merequetengue👍)

  ➢ Torraca Lautaro - 108813

  ➢ Negrotti Gianluca - 108184

  ➢ Marco Tosi - 107237

### > Imports

In [ ]:
import pandas as pd
import spacy
from langdetect import detect, DetectorFactory
from unidecode import unidecode
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from google.colab import files

### > Creacion de los datasets

In [ ]:
# subir archivo de train al colab (es muy pesado pasa subir a github)

train1 = pd.read_csv('https://raw.githubusercontent.com/lautitorraca/7506R-2C2023-GRUPO24/main/TP2/DataSets/train1.csv')
train2 = pd.read_csv('https://raw.githubusercontent.com/lautitorraca/7506R-2C2023-GRUPO24/main/TP2/DataSets/train2.csv')
train3 = pd.read_csv('https://raw.githubusercontent.com/lautitorraca/7506R-2C2023-GRUPO24/main/TP2/DataSets/train3.csv')
test = pd.read_csv('https://raw.githubusercontent.com/lautitorraca/7506R-2C2023-GRUPO24/main/TP2/DataSets/test.csv')

train_append = []

for i in [train1, train2, train3]:
  train_append.append(i)
train = pd.concat(train_append, ignore_index=True)
train = train.sort_values(by="ID").reset_index(drop=True)


- Muestra de train y test

In [ ]:
train.head()

,ID,review_es,sentimiento
0,0,Uno de los otros críticos ha mencionado que de...,positivo
1,1,Una pequeña pequeña producción.La técnica de f...,positivo
2,2,Pensé que esta era una manera maravillosa de p...,positivo
3,3,"Básicamente, hay una familia donde un niño peq...",negativo
4,4,"El ""amor en el tiempo"" de Petter Mattei es una...",positivo


In [ ]:
test.head()

,ID,review_es
0,60000,La mayor virtud de esta película es su existen...
1,60001,"No soy un experto cinéfilo, pero pocas veces m..."
2,60002,Si no eres un incondicional del humor estilo T...
3,60003,"No sé qué está pasando, si la gente se deja ll..."
4,60004,"Pero cuando amanece,y me quedo solo,siento en ..."


## > Limpieza

- Descarga y carag de diccionarios y recursos para la limpieza

In [ ]:
# Asegurar resultados consistentes de detección de idioma
DetectorFactory.seed = 0

# Cargar el modelo en español de Spacy
nlp = spacy.load('es_core_news_sm')

# Descargar recursos de NLTK
nltk.download('punkt')
nltk.download('stopwords')

# Cargar stopwords en español
spanish_stopwords = set(stopwords.words('spanish'))


- Definicion de funciones

In [ ]:

# Función para detectar si el texto es en español
def is_spanish(text):
    try:
        return detect(text) == 'es'
    except:
        # En caso de error en la detección, asumir que no es español
        return False

# Función para limpiar el texto
def clean_text(text):
    # Remover acentos y convertir a minúsculas
    text = unidecode(text).lower()
    # Tokenizar el texto y eliminar puntuaciones y números
    words = word_tokenize(text)
    words = [word for word in words if word.isalpha() and word not in spanish_stopwords]
    # Lematización con Spacy
    doc = nlp(' '.join(words))
    lemmas = [token.lemma_ for token in doc]
    return ' '.join(lemmas)


### > Limpieza en el dataset de Test

In [ ]:


# Se realiza primero la detección de idioma para evitar procesar texto que no sea español
test['is_spanish'] = test['review_es'].apply(is_spanish)

# Aquí estamos creando una nueva columna para los datos limpios
test['review_es_clean'] = test.apply(lambda row: clean_text(row['review_es']) if row['is_spanish'] else row['review_es'], axis=1)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


La limpieza del conjunto de datos de prueba se ha completado y se ha guardado en 'tu_archivo_test_limpio.csv'.


###TRAIN

In [ ]:

# Envolver la aplicación de la función is_spanish con tqdm para mostrar la barra de progreso
tqdm.pandas(desc="Detectando idioma")
train['is_spanish'] = train['review_es'].progress_apply(is_spanish)

# Envolver la aplicación de la función clean_text con tqdm para mostrar la barra de progreso
tqdm.pandas(desc="Limpiando texto")
train['review_es_clean'] = train.progress_apply(lambda row: clean_text(row['review_es']) if row['is_spanish'] else row['review_es'], axis=1)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Limpiando texto: 100%|██████████| 50000/50000 [22:42<00:00, 36.69it/s]


La limpieza del conjunto de datos de entrenamiento se ha completado y se ha guardado en 'tu_archivo_train_limpio.csv'.


### > Download de los datasets limpios

- Muestra de los datasets

  con el objetivo de corroborar que funciono la limpieza

In [ ]:
train.head()

,ID,review_es,sentimiento,is_spanish,review_es_clean
0,0,Uno de los otros críticos ha mencionado que de...,positivo,True,critico mencionado despu ver solo oz episodio ...
1,1,Una pequeña pequeña producción.La técnica de f...,positivo,True,pequén pequén tecnico filmacion incuestionable...
2,2,Pensé que esta era una manera maravillosa de p...,positivo,True,pense manera maravilloso pasar tiempo fin sema...
3,3,"Básicamente, hay una familia donde un niño peq...",negativo,True,basicamente familia nino pequeno jake pensar z...
4,4,"El ""amor en el tiempo"" de Petter Mattei es una...",positivo,True,amor tiempo petter mattei pelicular visualment...


In [ ]:
test.head()

,ID,review_es,is_spanish,review_es_clean
0,60000,La mayor virtud de esta película es su existen...,True,mayor virtud peliculo hecho poder jugar topico...
1,60001,"No soy un experto cinéfilo, pero pocas veces m...",True,experto cinefilo poco vez habia tanto juego sa...
2,60002,Si no eres un incondicional del humor estilo T...,True,si incondicional humor estilo tele termina cap...
3,60003,"No sé qué está pasando, si la gente se deja ll...",True,pasar si gente dejar llevar moda si verdad gus...
4,60004,"Pero cuando amanece,y me quedo solo,siento en ...",True,amanecer quedo solo siento fondoun mar vacio s...


- Prepearacion de los datasets para su descarga

In [ ]:

train[['ID','review_es_clean', 'sentimiento']].to_csv('train_limpio.csv', index=False)
test[['ID', 'review_es_clean']].to_csv('test_limpio.csv', index=False)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

- Descarga de los datasets

In [ ]:
try:
    files.download('train_limpio.csv')
    files.download('test_limpio.csv')
except ImportError:
    print("ERROR")